<a href="https://colab.research.google.com/github/AI-Junction/Personalized-Medicine/blob/master/PersonalizedMedRedCancerTreatVer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer



In [4]:
pwd = os.getcwd()
print(pwd)
dirname = os.path.join(pwd, "AllPersonalizedMedicineData")
print(dirname)


/content
/content/AllPersonalizedMedicineData


In [5]:
#for f in os.listdir('../input'):
for f in os.listdir(dirname):
    size_bytes = round(os.path.getsize(os.path.join(dirname, f))/ 1000, 2)
    size_name = ["KB", "MB"]
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    print(f.ljust(20) + str(s).ljust(7) + size_name[i])


FileNotFoundError: ignored